EURUSD_M15_201701020000_202206241445

In [ ]:
import pandas as pd
from xgboost import XGBRFClassifier

# Loading data and making it ready
name = "EURUSD_M15_201701020000_202206241445"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv/"

test_path = csv_path + "preprocessed/" + name + "/test.csv"


test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

test_date_time = test_df.pop(test_df.DateTime.name)

test_close = test_df.pop("Close")

test_open = test_df.Open[:]        #pop("Open")
test_high = test_df.High[:]        #pop("High")
test_low = test_df.Low[:]          #pop("Low")

train_df_rc = []
test_df_rc = []

relative_columns = [
    "BOLLINGER_HBAND",
    "BOLLINGER_LBAND",
    "BOLLINGER_MAVG",
    "DONCHIAN_CHANNEL_HBAND",
    "DONCHIAN_CHANNEL_LBAND",
    "DONCHIAN_CHANNEL_MBAND",
    "EMA_INDICATOR_12",
    "EMA_INDICATOR_26",
    "EMA_INDICATOR_50",
    "EMA_INDICATOR_100",
    "EMA_INDICATOR_200",
    "SMA_INDICATOR_12",
    "SMA_INDICATOR_26",
    "SMA_INDICATOR_50",
    "SMA_INDICATOR_100",
    "SMA_INDICATOR_200",
    "Open",
    "High",
    "Low",
    "ICHIMOKU_A",
    "ICHIMOKU_B",
    "ICHIMOKU_BASE_LINE",
    "ICHIMOKU_CONVERSION_LINE",
    "KELTNER_CHANNEL_HBAND",
    "KELTNER_CHANNEL_LBAND",
    "KELTNER_CHANNEL_MBAND",
    "NEGATIVE_VOLUME_INDEX",
    "VOLUME_WEIGHTED_AVERAGE_PRICE"
]

for column in relative_columns:
    train_df_rc.append(test_df.pop(column))
    
X_test = test_df

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
        

path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}'

model = XGBRFClassifier()
model.load_model(f"{path}/model.json")
p_test = model.predict(X_test)


df = X_test.copy()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = p_test
df["Close"] = test_close.tolist()
df["High"] = test_high.tolist()
df["Open"] = test_open.tolist()
df["Low"] = test_low.tolist()


In [ ]:
from backtesting import Strategy

def Signal():
    return df[:].Signal


class MyStrategy(Strategy):
    initsize = 0.02
    mysize = initsize
    c = 0

    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()

        if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
            if self.mysize != .32:
                self.mysize = self.mysize * 2
            else:
                self.c += 1

        elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
            if self.c > 0:
                # self.mysize=self.mysize / 2
                self.c -= 1
            else:
                self.mysize = self.initsize

        if self.signal1 == 0 and self.data.EMA_INDICATOR_100_C <= 1:
            sl1 = self.data.Close[-1] - (4 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            tp1 = self.data.Close[-1] + (12 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if self.signal1 == 2 and self.data.EMA_INDICATOR_100_C >= 1:
            sl1=self.data.Close[-1] + (4 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            tp1=self.data.Close[-1] - (12 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

from backtesting import Backtest

bt=Backtest(df[:], MyStrategy, cash=2_000, margin=1/100, commission=.0002,)
stat=bt.run()
stat

In [ ]:
bt.plot()

EURUSD_M15_201801020000_202206211100

In [ ]:
import pandas as pd
from xgboost import XGBRFClassifier

# Loading data and making it ready
name = "EURUSD_M15_201801020000_202206211100"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv/"

test_path = csv_path + "preprocessed/" + name + "/test.csv"


test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

test_date_time = test_df.pop(test_df.DateTime.name)

test_close = test_df.pop("Close")

test_open = test_df.Open[:]        #pop("Open")
test_high = test_df.High[:]        #pop("High")
test_low = test_df.Low[:]          #pop("Low")

train_df_rc = []
test_df_rc = []

relative_columns = [
    "BOLLINGER_HBAND",
    "BOLLINGER_LBAND",
    "BOLLINGER_MAVG",
    "DONCHIAN_CHANNEL_HBAND",
    "DONCHIAN_CHANNEL_LBAND",
    "DONCHIAN_CHANNEL_MBAND",
    "EMA_INDICATOR_12",
    "EMA_INDICATOR_26",
    "EMA_INDICATOR_50",
    "EMA_INDICATOR_100",
    "EMA_INDICATOR_200",
    "SMA_INDICATOR_12",
    "SMA_INDICATOR_26",
    "SMA_INDICATOR_50",
    "SMA_INDICATOR_100",
    "SMA_INDICATOR_200",
    "Open",
    "High",
    "Low",
    "ICHIMOKU_A",
    "ICHIMOKU_B",
    "ICHIMOKU_BASE_LINE",
    "ICHIMOKU_CONVERSION_LINE",
    "KELTNER_CHANNEL_HBAND",
    "KELTNER_CHANNEL_LBAND",
    "KELTNER_CHANNEL_MBAND",
    "NEGATIVE_VOLUME_INDEX",
    "VOLUME_WEIGHTED_AVERAGE_PRICE"
]

for column in relative_columns:
    train_df_rc.append(test_df.pop(column))
    
X_test = test_df

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
        

path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}'

model = XGBRFClassifier()
model.load_model(f"{path}/model.json")
p_test = model.predict(X_test)


df = X_test.copy()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = p_test
df["Close"] = test_close.tolist()
df["High"] = test_high.tolist()
df["Open"] = test_open.tolist()
df["Low"] = test_low.tolist()


In [ ]:
from backtesting import Strategy

def Signal():
    return df.Signal


class MyStrategy(Strategy):
    initsize = 0.02
    mysize = initsize
    c = 0

    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()

        if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
            if self.mysize != .32:
                self.mysize = self.mysize * 2
            else:
                self.c += 1

        elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
            if self.c > 0:
                # self.mysize=self.mysize / 2
                self.c -= 1
            else:
                self.mysize = self.initsize

        if self.signal1 == 0 and self.data.EMA_INDICATOR_200_C <= 1:
            sl1 = self.data.Close[-1] - (4 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            tp1 = self.data.Close[-1] + (12 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if self.signal1 == 2 and self.data.EMA_INDICATOR_200_C >= 1:
            sl1=self.data.Close[-1] + (4 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            tp1=self.data.Close[-1] - (12 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=2_000, margin=1/100, commission=.0002,)
stat=bt.run()
stat

In [ ]:
bt.plot()

EURUSD_M15_201901020600_202206241500

In [ ]:
import pandas as pd
from xgboost import XGBRFClassifier

# Loading data and making it ready
name = "EURUSD_M15_201901020600_202206241500"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv/"

test_path = csv_path + "preprocessed/" + name + "/test.csv"


test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

test_date_time = test_df.pop(test_df.DateTime.name)

test_close = test_df.pop("Close")

test_open = test_df.Open[:]        #pop("Open")
test_high = test_df.High[:]        #pop("High")
test_low = test_df.Low[:]          #pop("Low")

train_df_rc = []
test_df_rc = []

relative_columns = [
    "BOLLINGER_HBAND",
    "BOLLINGER_LBAND",
    "BOLLINGER_MAVG",
    "DONCHIAN_CHANNEL_HBAND",
    "DONCHIAN_CHANNEL_LBAND",
    "DONCHIAN_CHANNEL_MBAND",
    "EMA_INDICATOR_12",
    "EMA_INDICATOR_26",
    "EMA_INDICATOR_50",
    "EMA_INDICATOR_100",
    "EMA_INDICATOR_200",
    "SMA_INDICATOR_12",
    "SMA_INDICATOR_26",
    "SMA_INDICATOR_50",
    "SMA_INDICATOR_100",
    "SMA_INDICATOR_200",
    "Open",
    "High",
    "Low",
    "ICHIMOKU_A",
    "ICHIMOKU_B",
    "ICHIMOKU_BASE_LINE",
    "ICHIMOKU_CONVERSION_LINE",
    "KELTNER_CHANNEL_HBAND",
    "KELTNER_CHANNEL_LBAND",
    "KELTNER_CHANNEL_MBAND",
    "NEGATIVE_VOLUME_INDEX",
    "VOLUME_WEIGHTED_AVERAGE_PRICE"
]

for column in relative_columns:
    train_df_rc.append(test_df.pop(column))
    
X_test = test_df

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
        

path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}'

model = XGBRFClassifier()
model.load_model(f"{path}/model.json")
p_test = model.predict(X_test)


df = X_test.copy()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = p_test
df["Close"] = test_close.tolist()
df["High"] = test_high.tolist()
df["Open"] = test_open.tolist()
df["Low"] = test_low.tolist()


In [ ]:
from backtesting import Strategy

def Signal():
    return df.Signal


class MyStrategy(Strategy):
    initsize = 0.02
    mysize = initsize
    c = 0

    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()

        # if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
        #     if self.mysize != .32:
        #         self.mysize = self.mysize * 2
        #     else:
        #         self.c += 1

        # elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
        #     if self.c > 0:
        #         # self.mysize=self.mysize / 2
        #         self.c -= 1
        #     else:
        #         self.mysize = self.initsize

        if self.signal1 == 0 and self.data.EMA_INDICATOR_200_C <= 1:
            sl1 = self.data.Close[-1] - (4 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            tp1 = self.data.Close[-1] + (12 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if self.signal1 == 2 and self.data.EMA_INDICATOR_200_C >= 1:
            sl1=self.data.Close[-1] + (4 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            tp1=self.data.Close[-1] - (12 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=2_000, margin=1/100, commission=.0002,)
stat=bt.run()
stat

In [ ]:
bt.plot()

EURUSD_H1_201701020000_202206241400

In [ ]:
import pandas as pd
from xgboost import XGBRFClassifier

# Loading data and making it ready
name = "EURUSD_H1_201701020000_202206241400"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv/"

test_path = csv_path + "preprocessed/" + name + "/test.csv"


test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

test_date_time = test_df.pop(test_df.DateTime.name)

test_close = test_df.pop("Close")

test_open = test_df.Open[:]        #pop("Open")
test_high = test_df.High[:]        #pop("High")
test_low = test_df.Low[:]          #pop("Low")

train_df_rc = []
test_df_rc = []

relative_columns = [
    "BOLLINGER_HBAND",
    "BOLLINGER_LBAND",
    "BOLLINGER_MAVG",
    "DONCHIAN_CHANNEL_HBAND",
    "DONCHIAN_CHANNEL_LBAND",
    "DONCHIAN_CHANNEL_MBAND",
    "EMA_INDICATOR_12",
    "EMA_INDICATOR_26",
    "EMA_INDICATOR_50",
    "EMA_INDICATOR_100",
    "EMA_INDICATOR_200",
    "SMA_INDICATOR_12",
    "SMA_INDICATOR_26",
    "SMA_INDICATOR_50",
    "SMA_INDICATOR_100",
    "SMA_INDICATOR_200",
    "Open",
    "High",
    "Low",
    "ICHIMOKU_A",
    "ICHIMOKU_B",
    "ICHIMOKU_BASE_LINE",
    "ICHIMOKU_CONVERSION_LINE",
    "KELTNER_CHANNEL_HBAND",
    "KELTNER_CHANNEL_LBAND",
    "KELTNER_CHANNEL_MBAND",
    "NEGATIVE_VOLUME_INDEX",
    "VOLUME_WEIGHTED_AVERAGE_PRICE"
]

for column in relative_columns:
    train_df_rc.append(test_df.pop(column))
    
X_test = test_df

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
        

path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}'

model = XGBRFClassifier()
model.load_model(f"{path}/model.json")
p_test = model.predict(X_test)


df = X_test.copy()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = p_test
df["Close"] = test_close.tolist()
df["High"] = test_high.tolist()
df["Open"] = test_open.tolist()
df["Low"] = test_low.tolist()


In [ ]:
from backtesting import Strategy

def Signal():
    return df.Signal


class MyStrategy(Strategy):
    initsize = 0.02
    mysize = initsize
    c = 0

    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()

        if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
            if self.mysize != .32:
                self.mysize = self.mysize * 2
            else:
                self.c += 1

        elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
            if self.c > 0:
                # self.mysize=self.mysize / 2
                self.c -= 1
            else:
                self.mysize = self.initsize

        if self.signal1 == 0 and self.data.EMA_INDICATOR_200_C <= 1:
            sl1 = self.data.Close[-1] - (4 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            tp1 = self.data.Close[-1] + (12 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if self.signal1 == 2 and self.data.EMA_INDICATOR_200_C >= 1:
            sl1=self.data.Close[-1] + (4 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            tp1=self.data.Close[-1] - (12 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=2_000, margin=1/100, commission=.0002,)
stat=bt.run()
stat

In [ ]:
bt.plot()

EURUSD_H1_201801020000_202206210600

In [ ]:
import pandas as pd
from xgboost import XGBRFClassifier

# Loading data and making it ready
name = "EURUSD_H1_201801020000_202206210600"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv/"

test_path = csv_path + "preprocessed/" + name + "/test.csv"


test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

test_date_time = test_df.pop(test_df.DateTime.name)

test_close = test_df.pop("Close")

test_open = test_df.Open[:]        #pop("Open")
test_high = test_df.High[:]        #pop("High")
test_low = test_df.Low[:]          #pop("Low")

train_df_rc = []
test_df_rc = []

relative_columns = [
    "BOLLINGER_HBAND",
    "BOLLINGER_LBAND",
    "BOLLINGER_MAVG",
    "DONCHIAN_CHANNEL_HBAND",
    "DONCHIAN_CHANNEL_LBAND",
    "DONCHIAN_CHANNEL_MBAND",
    "EMA_INDICATOR_12",
    "EMA_INDICATOR_26",
    "EMA_INDICATOR_50",
    "EMA_INDICATOR_100",
    "EMA_INDICATOR_200",
    "SMA_INDICATOR_12",
    "SMA_INDICATOR_26",
    "SMA_INDICATOR_50",
    "SMA_INDICATOR_100",
    "SMA_INDICATOR_200",
    "Open",
    "High",
    "Low",
    "ICHIMOKU_A",
    "ICHIMOKU_B",
    "ICHIMOKU_BASE_LINE",
    "ICHIMOKU_CONVERSION_LINE",
    "KELTNER_CHANNEL_HBAND",
    "KELTNER_CHANNEL_LBAND",
    "KELTNER_CHANNEL_MBAND",
    "NEGATIVE_VOLUME_INDEX",
    "VOLUME_WEIGHTED_AVERAGE_PRICE"
]

for column in relative_columns:
    train_df_rc.append(test_df.pop(column))
    
X_test = test_df

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
        

path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}'

model = XGBRFClassifier()
model.load_model(f"{path}/model.json")
p_test = model.predict(X_test)


df = X_test.copy()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = p_test
df["Close"] = test_close.tolist()
df["High"] = test_high.tolist()
df["Open"] = test_open.tolist()
df["Low"] = test_low.tolist()


In [ ]:
from backtesting import Strategy

def Signal():
    return df.Signal


class MyStrategy(Strategy):
    initsize = 0.02
    mysize = initsize
    c = 0

    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()

        if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
            if self.mysize != .32:
                self.mysize = self.mysize * 2
            else:
                self.c += 1

        elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
            if self.c > 0:
                # self.mysize=self.mysize / 2
                self.c -= 1
            else:
                self.mysize = self.initsize

        if self.signal1 == 0 and self.data.EMA_INDICATOR_200_C >= 1:
            sl1 = self.data.Close[-1] - (4 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            tp1 = self.data.Close[-1] + (12 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if self.signal1 == 2 and self.data.EMA_INDICATOR_200_C <= 1:
            sl1=self.data.Close[-1] + (4 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            tp1=self.data.Close[-1] - (12 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=2_000, margin=1/100, commission=.0002,)
stat=bt.run()
stat

In [ ]:
bt.plot()

EURUSD_H1_201901020600_202206241500

In [ ]:
import pandas as pd
from xgboost import XGBRFClassifier

# Loading data and making it ready
name = "EURUSD_H1_201901020600_202206241500"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv/"

test_path = csv_path + "preprocessed/" + name + "/test.csv"


test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

test_date_time = test_df.pop(test_df.DateTime.name)

test_close = test_df.pop("Close")

test_open = test_df.Open[:]        #pop("Open")
test_high = test_df.High[:]        #pop("High")
test_low = test_df.Low[:]          #pop("Low")

train_df_rc = []
test_df_rc = []

relative_columns = [
    "BOLLINGER_HBAND",
    "BOLLINGER_LBAND",
    "BOLLINGER_MAVG",
    "DONCHIAN_CHANNEL_HBAND",
    "DONCHIAN_CHANNEL_LBAND",
    "DONCHIAN_CHANNEL_MBAND",
    "EMA_INDICATOR_12",
    "EMA_INDICATOR_26",
    "EMA_INDICATOR_50",
    "EMA_INDICATOR_100",
    "EMA_INDICATOR_200",
    "SMA_INDICATOR_12",
    "SMA_INDICATOR_26",
    "SMA_INDICATOR_50",
    "SMA_INDICATOR_100",
    "SMA_INDICATOR_200",
    "Open",
    "High",
    "Low",
    "ICHIMOKU_A",
    "ICHIMOKU_B",
    "ICHIMOKU_BASE_LINE",
    "ICHIMOKU_CONVERSION_LINE",
    "KELTNER_CHANNEL_HBAND",
    "KELTNER_CHANNEL_LBAND",
    "KELTNER_CHANNEL_MBAND",
    "NEGATIVE_VOLUME_INDEX",
    "VOLUME_WEIGHTED_AVERAGE_PRICE"
]

for column in relative_columns:
    train_df_rc.append(test_df.pop(column))
    
X_test = test_df

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
        

path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}'

model = XGBRFClassifier()
model.load_model(f"{path}/model.json")
p_test = model.predict(X_test)


df = X_test.copy()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = p_test
df["Close"] = test_close.tolist()
df["High"] = test_high.tolist()
df["Open"] = test_open.tolist()
df["Low"] = test_low.tolist()


In [ ]:
from backtesting import Strategy

def Signal():
    return df.Signal


class MyStrategy(Strategy):
    initsize = 0.02
    mysize = initsize
    c = 0

    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()

        if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
            if self.mysize != .32:
                self.mysize = self.mysize * 2
            else:
                self.c += 1

        elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
            if self.c > 0:
                # self.mysize=self.mysize / 2
                self.c -= 1
            else:
                self.mysize = self.initsize

        if self.signal1 == 0 and self.data.EMA_INDICATOR_200_C >= 1:
            sl1 = self.data.Close[-1] - (4 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            tp1 = self.data.Close[-1] + (12 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if self.signal1 == 2 and self.data.EMA_INDICATOR_200_C <= 1:
            sl1=self.data.Close[-1] + (4 * self.data.AVERAGE_TRUE_RANGE[-1]) + self.data.Spread[-1]*1e-4
            tp1=self.data.Close[-1] - (12 * self.data.AVERAGE_TRUE_RANGE[-1]) - self.data.Spread[-1]*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=2_000, margin=1/100, commission=.0002,)
stat=bt.run()
stat.to_csv(f"{path}/stat.csv")
stat

In [ ]:
bt.plot()

SP500m_H1_202009231400_202206241600

In [ ]:
import pandas as pd
from xgboost import XGBRFClassifier

# Loading data and making it ready
name = "SP500m_H1_202009231400_202206241600"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv/"

test_path = csv_path + "preprocessed/" + name + "/test.csv"


test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

test_date_time = test_df.pop(test_df.DateTime.name)

test_close = test_df.pop("Close")

test_open = test_df.Open[:]        #pop("Open")
test_high = test_df.High[:]        #pop("High")
test_low = test_df.Low[:]          #pop("Low")

train_df_rc = []
test_df_rc = []

relative_columns = [
    "BOLLINGER_HBAND",
    "BOLLINGER_LBAND",
    "BOLLINGER_MAVG",
    "DONCHIAN_CHANNEL_HBAND",
    "DONCHIAN_CHANNEL_LBAND",
    "DONCHIAN_CHANNEL_MBAND",
    "EMA_INDICATOR_12",
    "EMA_INDICATOR_26",
    "EMA_INDICATOR_50",
    "EMA_INDICATOR_100",
    "EMA_INDICATOR_200",
    "SMA_INDICATOR_12",
    "SMA_INDICATOR_26",
    "SMA_INDICATOR_50",
    "SMA_INDICATOR_100",
    "SMA_INDICATOR_200",
    "Open",
    "High",
    "Low",
    "ICHIMOKU_A",
    "ICHIMOKU_B",
    "ICHIMOKU_BASE_LINE",
    "ICHIMOKU_CONVERSION_LINE",
    "KELTNER_CHANNEL_HBAND",
    "KELTNER_CHANNEL_LBAND",
    "KELTNER_CHANNEL_MBAND",
    "NEGATIVE_VOLUME_INDEX",
    "VOLUME_WEIGHTED_AVERAGE_PRICE"
]

for column in relative_columns:
    train_df_rc.append(test_df.pop(column))
    
X_test = test_df

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
        

path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}'

model = XGBRFClassifier()
model.load_model(f"{path}/model.json")
p_test = model.predict(X_test)


df = X_test.copy()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = p_test
df["Close"] = test_close.tolist()
df["High"] = test_high.tolist()
df["Open"] = test_open.tolist()
df["Low"] = test_low.tolist()


In [ ]:
from backtesting import Strategy

def Signal():
    return df.Signal


class MyStrategy(Strategy):
    initsize = 0.02
    mysize = initsize
    c = 0

    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()

        if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
            if self.mysize != .32:
                self.mysize = self.mysize * 2
            else:
                self.c += 1

        elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
            if self.c > 0:
                # self.mysize=self.mysize / 2
                self.c -= 1
            else:
                self.mysize = self.initsize

        if self.signal1 == 0 and self.data.EMA_INDICATOR_200_C <= 1:
            sl1 = self.data.Close[-1] - (6 * self.data.AVERAGE_TRUE_RANGE[-1]) #- self.data.Spread[-1]*1e-4
            tp1 = self.data.Close[-1] + (24 * self.data.AVERAGE_TRUE_RANGE[-1])# + self.data.Spread[-1]*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if self.signal1 == 2 and self.data.EMA_INDICATOR_200_C >= 1:
            sl1=self.data.Close[-1] + (6 * self.data.AVERAGE_TRUE_RANGE[-1]) #+ self.data.Spread[-1]*1e-4
            tp1=self.data.Close[-1] - (24 * self.data.AVERAGE_TRUE_RANGE[-1])# - self.data.Spread[-1]*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=100_000, margin=1/100, commission=.0002)
stat=bt.run()
stat.to_csv(f"{path}/stat.csv")
stat

In [ ]:
bt.plot()

SP500m_M15_202009231430_202206241600

In [ ]:
import pandas as pd
from xgboost import XGBRFClassifier

# Loading data and making it ready
name = "SP500m_M15_202009231430_202206241600"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv/"

test_path = csv_path + "preprocessed/" + name + "/test.csv"


test_df = pd.read_csv(test_path)

test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

test_date_time = test_df.pop(test_df.DateTime.name)

test_close = test_df.pop("Close")

test_open = test_df.Open[:]        #pop("Open")
test_high = test_df.High[:]        #pop("High")
test_low = test_df.Low[:]          #pop("Low")

train_df_rc = []
test_df_rc = []

relative_columns = [
    "BOLLINGER_HBAND",
    "BOLLINGER_LBAND",
    "BOLLINGER_MAVG",
    "DONCHIAN_CHANNEL_HBAND",
    "DONCHIAN_CHANNEL_LBAND",
    "DONCHIAN_CHANNEL_MBAND",
    "EMA_INDICATOR_12",
    "EMA_INDICATOR_26",
    "EMA_INDICATOR_50",
    "EMA_INDICATOR_100",
    "EMA_INDICATOR_200",
    "SMA_INDICATOR_12",
    "SMA_INDICATOR_26",
    "SMA_INDICATOR_50",
    "SMA_INDICATOR_100",
    "SMA_INDICATOR_200",
    "Open",
    "High",
    "Low",
    "ICHIMOKU_A",
    "ICHIMOKU_B",
    "ICHIMOKU_BASE_LINE",
    "ICHIMOKU_CONVERSION_LINE",
    "KELTNER_CHANNEL_HBAND",
    "KELTNER_CHANNEL_LBAND",
    "KELTNER_CHANNEL_MBAND",
    "NEGATIVE_VOLUME_INDEX",
    "VOLUME_WEIGHTED_AVERAGE_PRICE"
]

for column in relative_columns:
    train_df_rc.append(test_df.pop(column))
    
X_test = test_df

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
        

path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}'

model = XGBRFClassifier()
model.load_model(f"{path}/model.json")
p_test = model.predict(X_test)


df = X_test.copy()
df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = p_test
df["Close"] = test_close.tolist()
df["High"] = test_high.tolist()
df["Open"] = test_open.tolist()
df["Low"] = test_low.tolist()


In [ ]:
from backtesting import Strategy

def Signal():
    return df.Signal


class MyStrategy(Strategy):
    initsize = 0.02
    mysize = initsize
    c = 0

    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next()

        if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
            if self.mysize != .32:
                self.mysize = self.mysize * 2
            else:
                self.c += 1

        elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
            if self.c > 0:
                # self.mysize=self.mysize / 2
                self.c -= 1
            else:
                self.mysize = self.initsize

        if self.signal1 == 0 and self.data.EMA_INDICATOR_200_C <= 1:
            sl1 = self.data.Close[-1] - (6 * self.data.AVERAGE_TRUE_RANGE[-1]) #- self.data.Spread[-1]*1e-4
            tp1 = self.data.Close[-1] + (24 * self.data.AVERAGE_TRUE_RANGE[-1])# + self.data.Spread[-1]*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if self.signal1 == 2 and self.data.EMA_INDICATOR_200_C >= 1:
            sl1=self.data.Close[-1] + (6 * self.data.AVERAGE_TRUE_RANGE[-1]) #+ self.data.Spread[-1]*1e-4
            tp1=self.data.Close[-1] - (24 * self.data.AVERAGE_TRUE_RANGE[-1])# - self.data.Spread[-1]*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=100_000, margin=1/100, commission=.0002)
stat=bt.run()
stat.to_csv(f"{path}/stat.csv")
stat

In [ ]:
bt.plot()